In [ ]:
import labrotation.file_handling as fh
import pims_nd2
from datetime import datetime as dt
from datetime import timedelta
import pytz
import datadoc_util as ddu
import os

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

In [ ]:
if "DATA_DOCU_FOLDER" in env_dict.keys():
    data_docu_folder = env_dict["DATA_DOCU_FOLDER"]
else:
    data_docu_folder = fh.open_dir("Open Data Documentation folder")
print(data_docu_folder)

In [ ]:
ddoc = ddu.DataDocumentation(data_docu_folder)
ddoc.loadDataDoc()

In [ ]:
tzone_utc = pytz.utc
tzone_local = pytz.timezone('Europe/Berlin')

In [ ]:
nd2 = fh.open_file("Choose nd2 file!")

In [ ]:
print(nd2)

In [ ]:
uuid = ddoc.getUUIDForFile(os.path.split(nd2)[-1])
print(uuid)

In [ ]:
segments_df = ddoc.getSegmentsForUUID(uuid)

In [ ]:
segments_df = segments_df[segments_df["interval_type"] == "sz"]
print(segments_df)

In [ ]:
nd2_data = pims_nd2.ND2_Reader(nd2)

In [ ]:
t0 = tzone_utc.localize(nd2_data.metadata["time_start_utc"])

In [ ]:
for i_sz, row in segments_df.iterrows():
    i_frame = row.frame_begin - 1
    dt_ms = timedelta(milliseconds=nd2_data[i_frame].metadata["t_ms"])
    t = t0 + dt_ms
    t.isoformat()
    print(f"Seizure #{i_sz}: {i_frame} - {t.astimezone(tzone_local).isoformat()}")